In [1]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from warnings import warn

In [2]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search/?f_TPR=r86400&geoId=103644278&keywords=data%20analyst&location=United%20States"
no_of_jobs = 25

In [3]:
#driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_path = r'/Users/vidhigandhi/Desktop/chromedriver' #path from 'which chromedriver'
driver = webdriver.Chrome(executable_path=chrome_path)

In [4]:
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [6]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [7]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [8]:
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

In [9]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

25
25
25
25
25
25
25
25
25
25


In [10]:
# creating a dataframe
linkedin_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

# cleaning description column
linkedin_data['Description'] = linkedin_data['Description'].str.replace('\n',' ')

print(linkedin_data.info())
linkedin_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
Job ID          25 non-null object
Date            25 non-null object
Company Name    25 non-null object
Post            25 non-null object
Location        25 non-null object
Description     25 non-null object
Level           25 non-null object
Type            25 non-null object
Function        25 non-null object
Industry        25 non-null object
dtypes: object(10)
memory usage: 2.1+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2374994845,2021-01-21,Apple,Data Analyst,"Cupertino, CA",Summary The people here at Apple don’t just b...,Not Applicable,Full-time,Information Technology,Consumer Electronics
1,2357705870,2021-01-06,Twitter,Data Analyst - Revenue Intelligence,"San Francisco, CA",Sales Finance provides insight into Twitter’s ...,Not Applicable,Full-time,Information Technology,Internet
2,2388928210,2021-01-21,PETADATA,Data Analyst,"San Jose, CA",PETADATA is looking for a Data Analyst to work...,Entry level,Full-time,Information Technology,Information Technology and Services
3,2351004209,2020-12-31,PayPal,Data Analyst,"Scottsdale, AZ",Fueled by a fundamental belief that having acc...,Not Applicable,Full-time,Information Technology,Computer SoftwareFinancial ServicesInternet
4,2365166946,2020-12-19,Cisco,Data Analyst,"San Jose, CA",What You'll Do The Growth Analytics team will...,Not Applicable,Full-time,Information Technology,Computer HardwareComputer SoftwareComputer Net...


In [11]:
linkedin_data.to_csv('LinkedIn Job Data.csv', index=0)